In [ ]:
"""
Navigate to Your Project Directory: cd E:\Projects\llamaRiskDataNotebooks

Activate the Virtual Environment: .\env\Scripts\activate.bat

"""

In [1]:
from requests import get, post
from web3 import Web3, HTTPProvider
from icecream import ic
import numpy as np
import pandas as pd
import time
import json
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from pandas import json_normalize
import sys
from dotenv import load_dotenv
import os

In [14]:
# Add the directory containing your module to the Python path
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

# Import your functions
from functions01 import add, subtract, multiply, divide, query_smart_contract
from uniV3Pricing import get_arcadia_account_nft_position, get_account_data

In [4]:
# Explicitly specify the path to .env file using forward slashes
load_dotenv(dotenv_path='E:/Projects/llamaRiskDataNotebooks/.env')

# Verify that the environment variable is loaded correctly
alchemy_key = os.getenv('ALCHEMY_KEY')

# Create the Web3 provider using the Alchemy URL
w3 = Web3(HTTPProvider(f"https://base-mainnet.g.alchemy.com/v2/{alchemy_key}")) # BASE
# w3 = Web3(HTTPProvider(f"https://eth-mainnet.g.alchemy.com/v2/{alchemy_key}")) # ETHEREUM

# Check connection
print(f"Web3 is connected: {w3.is_connected()}")


Web3 is connected: True


In [11]:
latest_block = w3.eth.block_number
print(f"Latest block number: {latest_block}")

# Check Web3 connection
print(f"Web3 is connected: {w3.is_connected()}")

# Example contract address and ABI (you should replace these with actual values)
contract_address = "0x1d3e2c623c7d1c2cc0329ee241a62c47bc213145"
abi = [{
    "inputs": [],
    "name": "generateAssetData",
    "outputs": [
        {
            "internalType": "address[]",
            "name": "assetAddresses",
            "type": "address[]",
        },
        {"internalType": "uint256[]", "name": "assetIds", "type": "uint256[]"},
        {"internalType": "uint256[]", "name": "assetAmounts", "type": "uint256[]"},
    ],
    "stateMutability": "view",
    "type": "function",
}]

# Query the 'generateAssetData' function (example)
result = query_smart_contract(w3, contract_address, abi, 'generateAssetData')
print(result)

Latest block number: 16838571
Web3 is connected: True
[['0x4200000000000000000000000000000000000006', '0xd9aAEc86B65D86f6A7B5B1b0c42FFA531710b6CA', '0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913', '0x03a520b32C04BF3bEEf7BEb72E919cf822Ed34f1', '0x827922686190790b37229fd06084350E74485b72'], [0, 0, 0, 514751, 25926], [645786837953691, 658636, 5252454, 1, 1]]


In [18]:
assets = ['0x2Ae3F1Ec7F1F5012CFEab0185bfc7aa3cf0DEc22',
       '0x4200000000000000000000000000000000000006',
       '0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913',
       '0xd9aAEc86B65D86f6A7B5B1b0c42FFA531710b6CA',
       '0xc1CBa3fCea344f92D9239c08C0568f6F2F0ee452',
       '0x940181a94A35A4569E4529A3CDfB74e38FD98631',
       '0x50c5725949A6F0c72E6C4a641F24049A917DB0Cb',
       '0xB6fe221Fe9EeF5aBa221c348bA20A1Bf5e73624c']
decimal_abi = [{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"}]
decimal_list = []

for i in assets:
    decimal = query_smart_contract(w3, i, decimal_abi, 'decimals')
    decimal_list.append(decimal)
    print(i, decimal)

print(decimal_list)

0x2Ae3F1Ec7F1F5012CFEab0185bfc7aa3cf0DEc22 18
0x4200000000000000000000000000000000000006 18
0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913 6
0xd9aAEc86B65D86f6A7B5B1b0c42FFA531710b6CA 6
0xc1CBa3fCea344f92D9239c08C0568f6F2F0ee452 18
0x940181a94A35A4569E4529A3CDfB74e38FD98631 18
0x50c5725949A6F0c72E6C4a641F24049A917DB0Cb 18
0xB6fe221Fe9EeF5aBa221c348bA20A1Bf5e73624c 18
[18, 18, 6, 6, 18, 18, 18, 18]


In [16]:
def get_account_data(account_address, w3=w3):
    
    account_abi = [{
        "inputs": [],
        "name": "generateAssetData",
        "outputs": [
            {
                "internalType": "address[]",
                "name": "assetAddresses",
                "type": "address[]",
            },
            {"internalType": "uint256[]", "name": "assetIds", "type": "uint256[]"},
            {"internalType": "uint256[]", "name": "assetAmounts", "type": "uint256[]"},
        ],
        "stateMutability": "view",
        "type": "function",
    }]
    
    try:
        # Pool contract
        contract = w3.eth.contract(
            address=Web3.to_checksum_address(account_address), #0xb2DF87b16682435bdf29494fEA44dF5374F22f3E
            abi=account_abi
        )
        data = contract.functions.generateAssetData().call()
        return data
    except Exception as e:
        return f'Error fetching pool_oracle: {e}'
    
get_account_data("0x1d3e2c623c7d1c2cc0329ee241a62c47bc213145")

[['0x4200000000000000000000000000000000000006',
  '0xd9aAEc86B65D86f6A7B5B1b0c42FFA531710b6CA',
  '0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913',
  '0x03a520b32C04BF3bEEf7BEb72E919cf822Ed34f1',
  '0x827922686190790b37229fd06084350E74485b72'],
 [0, 0, 0, 514751, 25926],
 [645786837953691, 658636, 5252454, 1, 1]]

In [17]:
x = get_account_data("0x1d3e2c623c7d1c2cc0329ee241a62c47bc213145",w3)
print(x)

[['0x4200000000000000000000000000000000000006', '0xd9aAEc86B65D86f6A7B5B1b0c42FFA531710b6CA', '0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913', '0x03a520b32C04BF3bEEf7BEb72E919cf822Ed34f1', '0x827922686190790b37229fd06084350E74485b72'], [0, 0, 0, 514751, 25926], [645786837953691, 658636, 5252454, 1, 1]]


"Error fetching pool_oracle: Could not format invalid value {'inputs': [], 'name': 'generateAssetData', 'outputs': [{'internalType': 'address[]', 'name': 'assetAddresses', 'type': 'address[]'}, {'internalType': 'uint256[]', 'name': 'assetIds', 'type': 'uint256[]'}, {'internalType': 'uint256[]', 'name': 'assetAmounts', 'type': 'uint256[]'}], 'stateMutability': 'view', 'type': 'function'} as field 'abi'"

In [ ]:
get_arcadia_account_nft_position(x, w3=w3)